In [1]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 2000)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms
plt.style.use('fivethirtyeight')

In [2]:
# Import relevant dataframes:

main_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Main_Data\Main_Table_10182019.csv")
print(len(main_table))
# faculty_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Faculty_Data\Faculty_Table_08272019.csv")
# print(len(faculty_table))
# skill_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Skills_Data\Skill_Table_06072019.csv")
# print(len(skill_table))

3957167


In [3]:
def title_swap(string):
    dictionary = {'Community and Social Services':'Counseling and Religious Life',
                  'Customer and Client Support':'Online Support and University Information',
                  'Hospitality, Food, and Tourism':'Event Management and Hospitality', 
                  'Planning and Analysis':'Analysis', 
                  'Curriculum and Instructional Designer / Developer':'Curriculum and Instructional Designer', 
                  'Special Education Teacher':'Accessibility and Disability Services', 
                  'Teaching Assistant':'Faculty Support', 
                  'Tutor':'Academic Tutor',
                  'Clerical and Administrative':'Administrative'}
    if(string in dictionary):
        return dictionary[string]
    else:
        return string

In [5]:
table = main_table[main_table['Occupation'].str.contains('Dean', na=False)]
table['Occupation'].value_counts()

Dean of Academic Division           19507
Dean of Academic Affairs             8034
Associate Dean, Higher Education     6425
Dean of Students                     3101
Dean (General)                       1679
Name: Occupation, dtype: int64

In [6]:
main_table['IPEDS Institution Name'].value_counts()

University of California-System Administration Central Office                             149244
Carolinas College of Health Sciences                                                       63234
University of Maryland, Baltimore                                                          41906
University of Colorado Denver/Anschutz Medical Campus                                      40965
University of Washington-Seattle Campus                                                    39707
University of Michigan-Ann Arbor                                                           39009
University of Wisconsin Colleges                                                           36312
Vanderbilt University                                                                      35302
University of Southern California                                                          29181
University of Minnesota-Twin Cities                                                        28706
Columbia University in the Cit

In [18]:
# Skills to look for:
"""
# Requirements
Policy analysis                     [yes, cluster]
Policy proposal                     [yes, skill]
Policy research                     [yes, skill]
Policy evaluation                   [yes, skill]
Policy development                  [yes, skill]
Policy implementation               [yes, skill]
Public policy development           [yes, skill]
Social work                         [yes, skill & cluster]
Social services                     [yes, skill]
Social services industry knowledge  [yes, skill & cluster]
Behavioral health                   [yes, skill]
    
# Supplememts
Data management                     [yes, cluster]
Health data management              [yes, cluster]
Research                            [yes, skill]
Health research                     [yes, skill]
Survey research                     [yes, skill]
Research design                     [yes, skill]
Qualitative research                [yes, skill]
Regression analysis                 [yes, skill]
"""

column = 'Skill Cluster Name'
item = 'regression analysis'

table = skill_table[skill_table[column].str.contains(item, case=False, na=False)]
values = pd.DataFrame(table[column].value_counts()).reset_index().sort_values('index')
values

,index,Skill Cluster Name


In [ ]:
# List of Primary SP Skills
"""
Social Work
Social Policy
Behavioral Health
Social Services Industry Knowledge 
"""

# List of Secondary SP Skills


In [9]:
skill = 'Social Work'

category = 'Skill Name'

mask = skill_table[skill_table[category]==skill][['Job ID']]

table = skill_table.merge(mask, on = 'Job ID', how = 'inner')

table['Skill Name'].value_counts()

Social Work                                                              11581
Communication Skills                                                      3636
Case Management                                                           3015
Mental Health                                                             2914
Crisis Intervention                                                       2769
Teamwork / Collaboration                                                  2680
Discharge Planning                                                        2520
Research                                                                  2482
Treatment Planning                                                        2437
Patient Care                                                              2143
Teaching                                                                  2017
Social Services                                                           2010
Planning                                            

In [6]:
mask = skill_table[(skill_table['Skill Name']=='Social Work')|
                   (skill_table['Skill Name']=='Social Services')
                   ][['Job ID']]
# &
#                    (skill_table['Skill Name']=='Research')

# to add another filter: | (skill_table['Skill Name']=='')
mask = mask.drop_duplicates('Job ID')
# print(len(mask))

table = main_table.merge(mask, on='Job ID', how='inner')
print('Number of total Jobs:\n\t' + str(len(table)))

table['Career Area'] = table['Career Area'].apply(title_swap)
table['Occupation'] = table['Occupation'].apply(title_swap)

table = table[(table['Minimum EDU Requirements']>=18)]

print('Number of Jobs with MA or Above:\n\t' + str(len(table)))

mask = pd.DataFrame(table['Occupation'].value_counts()).reset_index().rename(columns={'Occupation':'count', 'index':'Occupation'})
mask = mask[mask['count']>=10][['Occupation']]
table = table.merge(mask, on='Occupation', how='inner')
# print('Number of total Jobs:\n\t', len(table))

mask = faculty_table[(faculty_table['Faculty']==0)&(faculty_table['Post-Doctoral']==0)]['Job ID']
table = table.merge(mask, on='Job ID', how='inner')
print('Number of non-faculty Jobs:\n\t' + str(len(table)))



Number of total Jobs:
	33146
Number of Jobs with MA or Above:
	10342
Number of non-faculty Jobs:
	8151


In [ ]:
# Occupational Breakdowns

In [31]:
pd.DataFrame(table[['Job ID', 'Career Area', 'Occupation']].groupby(['Career Area', 'Occupation']).count()).rename(columns={'Job ID':'Count'})

Count
Career Area                        Occupation                                          
Administrative                     Administrative Manager                            19
                                   Office / Administrative Assistant                 15
Analysis                           Business / Management Consultant                  15
                                   Business Analyst (General)                        21
                                   Junior Business Analyst                           13
Business Management and Operations Director of Program Management                    10
                                   Education Program Manager                         45
                                   Executive Director                                10
                                   General Manager                                   15
                                   Program Coordinator                               13
                                   Project Coordinator                               14
                                   Project Manager (General)                         32
Counseling and Religious Life      Academic / Guidance Counselor                     37
                                   Career Counselor                                  20
                                   Family / School / General Social Worker         3263
                                   Medical / Clinical Social Worker                1529
                                   Mental Health / Behavioral Counselor             391
                                   Social / Human Service Assistant                 277
                                   Social / Human Services Manager                  148
Education and Training             Accessibility and Disability Services             10
                                   Administrator, Higher Education                  108
                                   Admissions Director                               14
                                   Adult Basic Education / Literacy Instructor       10
                                   Associate Dean, Higher Education                  10
                                   Curriculum and Instructional Designer             21
                                   Dean of Academic Division                        169
                                   Director of Student Affairs                       19
                                   Elementary School Teacher                         12
                                   Elementary and Secondary School Administrator     10
                                   Librarian                                         27
                                   Preschool / Childcare Teacher                     13
                                   Program Director, Higher Education                36
                                   Student Affairs Administrator                     22
                                   Vocational Education Instructor                   10
Finance                            Financial Manager (General)                       16
Health Care including Nursing      Audiologist                                       10
                                   Care Coordinator                                   9
                                   Clinical Case Manager                             62
                                   Clinical Coordinator                              21
                                   Clinical Director                                  9
                                   Clinical Nurse Specialist                         15
                                   Clinical Supervisor / Manager                     43
                                   Director of Surgical Services                     18
                                   Family / Behavioral Therapist                     53
                                  

In [32]:
s = pd.DataFrame(table[['Job ID']].merge(skill_table, on='Job ID', how='inner')['Skill Name'].value_counts())

In [33]:
s

,Skill Name
Social Work,5929
Social Services,3502
Communication Skills,3171
Mental Health,2501
Teamwork / Collaboration,2420
Treatment Planning,2304
Crisis Intervention,2266
Case Management,1968
Discharge Planning,1941
Research,1795


In [7]:
table[table['Occupation']=='Chief Executive Officer'][['Job ID']].merge(skill_table)

,Job ID,Year,Job Title,Occupation,Career Area,Minimum EDU Requirements,Maximum EDU Requirements,Minimum Experience Requirements,Annual Salary,Metropolitan Statistical Area,State,IPEDS Institution Name,IPEDS Sector Name,R1,4-year,2-year,Public,Private,BEA_Zone
26715,429133496,2014,Chief Operating Officer - Ubhc,Chief Executive Officer,Business Management and Operations,18,21,10.00,-999.0,"New York-Newark-Jersey City, NY-NJ-PA",New Jersey,Rutgers University-New Brunswick,"Public, 4-year or above",1,1,0,1,0,Mideast
26716,432860317,2014,Chief Operating Officer - Ubhc,Chief Executive Officer,Business Management and Operations,18,21,10.00,-999.0,"New York-Newark-Jersey City, NY-NJ-PA",New Jersey,Rutgers University-New Brunswick,"Public, 4-year or above",1,1,0,1,0,Mideast
26717,37853951669,2014,Chief Executive Officer,Chief Executive Officer,Business Management and Operations,18,-999,10.00,-999.0,"Bridgeport-Stamford-Norwalk, CT",Connecticut,University of New England,"Private not-for-profit, 4-year or above",0,1,0,0,1,New England
26718,37884712137,2015,Coordinator - Health & Welfare Planning And Fi...,Chief Executive Officer,Business Management and Operations,16,18,6.00,74100.0,"San Francisco-Oakland-Hayward, CA",California,University of California-System Administration...,Administrative Unit,0,1,0,1,0,Far West
26719,37889441542,2015,Chief Executive Officer,Chief Executive Officer,Business Management and Operations,-999,-999,-83.25,-999.0,"Flagstaff, AZ",Arizona,Northern Arizona University,"Public, 4-year or above",0,1,0,1,0,Southwest
26720,38068995183,2016,Chief Executive Officer Psychiatric Hospital,Chief Executive Officer,Business Management and Operations,18,-999,5.00,-999.0,"Nashville-Davidson--Murfreesboro--Franklin, TN",Tennessee,Vanderbilt University,"Private not-for-profit, 4-year or above",1,1,0,0,1,Southeast
26721,38080510082,2016,Chief Operating Officer Of University Of Healt...,Chief Executive Officer,Business Management and Operations,18,-999,10.00,-999.0,"Salt Lake City, UT",Utah,University of Utah,"Public, 4-year or above",1,1,0,1,0,Rocky Mountiains
26722,38151937820,2016,"Chief Operating Officer, Uchc",Chief Executive Officer,Business Management and Operations,21,21,-83.25,-999.0,"New York-Newark-Jersey City, NY-NJ-PA",New Jersey,Rutgers University-New Brunswick,"Public, 4-year or above",1,1,0,1,0,Mideast
26723,38199046553,2017,Chief Executive Officer,Chief Executive Officer,Business Management and Operations,16,18,5.00,-999.0,"Rochester, MN",Minnesota,NaN,"Public, 4-year or above",0,1,0,1,0,Plains
26724,38207632200,2017,Chief Executive Officer,Chief Executive Officer,Business Management and Operations,21,-999,10.00,-999.0,"Rochester, MN",Minnesota,Loyola University Chicago,"Private not-for-profit, 4-year or above",0,1,0,0,1,Plains


In [ ]:
skills = table[['Job ID']].merge(skill_table, on='Job ID', how='inner')

In [7]:
index = faculty_table[faculty_table['Faculty']==1]
table = main_table[main_table['Job Title'].str.contains('post-doc', case=False)|main_table['Job Title'].str.contains('post doc', case=False)]

In [10]:
final = index[['Job ID', 'Post-Doctoral']].merge(table, on='Job ID', how = 'inner')
display(len(final))
final = final[final['Post-Doctoral']==0]
display(len(final))

245

149

In [11]:
final['Job Title'].value_counts()

Adjunct Research Assistant/Associate/Post-Doctoral                                                                                   5
Post-Doctoral/Junior Faculty                                                                                                         5
Post-Doctoral Junior Faculty                                                                                                         4
Post-Doctoral Researcher/Research Assistant Professor                                                                                4
Post-Doctoral Researcher Research Assistant Professor                                                                                3
Lecturer/Post-Doctoral Scholar                                                                                                       3
Research Assistant Professor/Post-Doctoral Researcher                                                                                3
Post-Doctoral Faculty - Civil Engineering              

In [14]:
table = main_table[main_table['Job Title'].str.contains('post-doc', case=False)|main_table['Job Title'].str.contains('post doc', case=False)]
table = table.merge(faculty_table[['Job ID', 'Faculty', 'Post-Doctoral']], on='Job ID', how='inner')

In [18]:
main_table[main_table['Occupation'].str.contains('fundraising', case=False)]['Occupation'].value_counts()

Fundraising Manager                     35254
Fundraising / Development Specialist     7758
Name: Occupation, dtype: int64

In [15]:
table.groupby(['Faculty', 'Post-Doctoral']).count()

Unnamed: 0  Unnamed: 0.1  Job ID   Year  Job Title  \
Faculty Post-Doctoral                                                       
0       0                     101           101     101    101        101   
        1                   35992         35992   35992  35992      35992   
1       0                     149           149     149    149        149   
        1                      96            96      96     96         96   

                       Occupation  Career Area  Minimum EDU Requirements  \
Faculty Post-Doctoral                                                      
0       0                     101          101                       101   
        1                   35992        35992                     35992   
1       0                     149          149                       149   
        1                      96           96                        96   

                       Maximum EDU Requirements  \
Faculty Post-Doctoral                             
0       0                                   101   
        1                                 35992   
1       0                                   149   
        1                                    96   

                       Minimum Experience Requirements  Annual Salary  \
Faculty Post-Doctoral                                                   
0       0                                          101            101   
        1                                        35992          35992   
1       0                                          149            149   
        1                                           96             96   

                       Metropolitan Statistical Area  State  \
Faculty Post-Doctoral                                         
0       0                                        101    101   
        1                                      35992  35992   
1       0                                        149    149   
        1                                         96     96   

                       IPEDS Institution Name  IPEDS Sector Name     R1  \
Faculty Post-Doctoral                                                     
0       0                                  95                101    101   
        1                               31856              35992  35992   
1       0                                  42                 44    149   
        1                                  95                 96     96   

                       4-year  2-year  Public  Private  BEA_Zone  
Faculty Post-Doctoral                                             
0       0                 101     101     101      101       101  
        1               35992   35992   35992    35992     35992  
1       0                 149     149     149      149       149  
        1                  96      96      96       96        96

36338

In [7]:
id = faculty_table[(faculty_table['Faculty']==1)&(faculty_table['Post-Doctoral']==1)]

main_table.merge(id[['Job ID']], on='Job ID', how='inner')['Job Title'].value_counts()

Post-Doctoral/Staff Scientist/Assistant Professor                                                                                 7
Post-Doctoral, Research Scientist, Research Assistant Prof, Research Associate Professor, Research Professor                      4
Post-Doctoral Research Professor - Mechanical Engineering                                                                         4
Post-Doctoral, Research Scientist, Research Assistant Prof, Research Associate Professor, Research Professor*                     3
Post-Doctoral, Research Scientist, Research Assistant Professor, Research Associate Prof, Research Professor                      3
Post-Doctoral, Research Scientist, Research Assistant Professor, Research Associate Prof, Research Professor*                     3
Post-Doctoral Scholar - Professor Chang Liu Laboratory                                                                            3
Engineering Pools Post-Doctoral, Research Scientist, Research Assistant Prof

In [3]:
len(main_table[main_table['Job Title'].str.contains('open rank', case=False, na=False)])

10842

In [4]:
len(main_table[main_table['Job Title'].str.contains('open-rank', case=False, na=False)])

238

In [6]:
len(main_table[main_table['Job Title'].str.contains('open', case=False, na=False) 
    & main_table['Job Title'].str.contains('rank', case=False, na=False)])

11480

In [ ]:
faculty_table[faculty_table['Other humanities and arts']==1].merge(main_table['Job ID'])

In [ ]:
IS_main = pd.read_csv(r"G:\IS_Data\IS_Main.csv")
IS_skill = pd.read_csv(r"G:\IS_Data\IS_Skill.csv")

In [ ]:
IS_main.drop(columns=['Job Title', 'Minimum EDU Requirements', 'Minimum Experience Requirements', 'Unnamed: 0']).to_csv(r"G:\IS_Data\IS_Main.csv")

In [ ]:
IS_skill

In [ ]:
full_table = main_table.merge(faculty_table[['Job ID', 'Faculty']], on='Job ID', how='inner')

full_table = full_table[full_table['Faculty']==0]
full_table = full_table[full_table['Year']==2017]
full_table = full_table[(full_table['Minimum EDU Requirements']==21)]

full_table = full_table[(full_table['Career Area']!='Health Care including Nursing')&(full_table['Career Area']!='na')]

# These Job Titles are omitted because they are likely to hold JD or MD rather than PhD

full_table = full_table[~(full_table['Career Area'].str.contains('attorney', case=False, na=False)) &
                        ~(full_table['Occupation'].str.contains('attorney', case=False, na=False)) &
                        ~(full_table['Job Title'].str.contains('attorney', case=False, na=False)) & 
                        ~(full_table['Job Title'].str.contains('title ix', case=False, na=False)) &
                        ~(full_table['Job Title'].str.contains('director of advancement', case=False, na=False)) &
                        ~(full_table['Job Title'].str.contains('finance manager', case=False, na=False)) & 
                        ~(full_table['Job Title'].str.contains('financial manager', case=False, na=False)) &
                        ~(full_table['Occupation'].str.contains('financial manager', case=False, na=False)) &
                        ~(full_table['Career Area'].str.contains('financial manager', case=False, na=False)) &
                        ~(full_table['IPEDS Institution Name'].str.contains('law', case=False, na=False)) & 
                        ~(full_table['IPEDS Institution Name'].str.contains('medical', case=False, na=False))]

# Remove Elementary school teachers for now, financial occupations as well


In [ ]:
full_table = full_table[full_table['Career Area']=='Education and Training']

full_table = full_table[~full_table['Job Title'].str.contains('professor', case=False, na=False)]
full_table = full_table[~(full_table['Occupation']=='College Professor/Instructor')]

full_table.sort_values(by=['Occupation', 'IPEDS Institution Name'])

In [ ]:
full_table = full_table[full_table['Career Area'].str.contains('Information Technology', case=False, na=False)]

full_table['Occupation'].value_counts()

full_table[(full_table['Occupation']=='Computer Scientist / Researcher')]['Job Title'].value_counts()[:10]

In [ ]:
full_table[full_table['Job Title']=='Law Admissions Counselor']

In [ ]:
pd.Series(' '.join(full_table['Job Title']).lower().split()).value_counts()[:100]

In [ ]:
#############################################################
# Growth of Non-Faculty Job Postings by Career area 2007-2017
#############################################################

# Separate out all columns relevant to this evaluation:
full_table = main_table[['Job ID', 'Year', 'R1', 'Career Area', '4-year', '2-year']].merge(
    faculty_table[['Job ID', 'Faculty', 'Post-Doctoral']], how="inner", on="Job ID")


# Drop non-Faculty
full_table = full_table[(full_table['Faculty'] == 0) & (full_table['Post-Doctoral'] != 1)]

# Drop Career Areas with fewer than 1000 observation in 2017
full_table = full_table[(full_table['Career Area']!='na')]

# Restrict years of window
full_table = full_table[(full_table['Year']==2007)|(full_table['Year'] == 2017)]
# Restrict to R1 Institutions
# full_table = full_table[full_table['R1'] == 1]

table_07 = pd.DataFrame(full_table[full_table['Year']==2007]['Career Area'].value_counts()).reset_index().rename(
    columns={'Career Area':'count_07', 'index':'Career Area'})

table_17 = pd.DataFrame(full_table[full_table['Year']==2017]['Career Area'].value_counts()).reset_index().rename(
    columns={'Career Area':'count_17', 'index':'Career Area'})

final_table = table_07.merge(table_17, on='Career Area', how='inner')

final_table = final_table[final_table['count_17']>999]

final_table['growth'] = round(np.true_divide(final_table['count_17']-final_table['count_07'], final_table['count_07'])*100, 2)

final_table.sort_values(by='growth', ascending=False)

In [ ]:
source_skill_table

In [ ]:
salary = main_table[main_table['Annual Salary']>0]

# pd.DataFrame(salary['IPEDS Institution Name'].value_counts()).reset_index().sort_values(by='index')

# salary[salary['IPEDS Institution Name']=='Brandeis University']

salary.merge(faculty_table[['Job ID', 'Faculty']], on='Job ID', how='inner')['Faculty'].value_counts()

In [ ]:
main_table[main_table['Annual Salary']>100000]

In [8]:
faculty_table.columns

Index(['Unnamed: 0', 'Job ID', 'Year', 'Diversity and Inclusion',
       'Post-Doctoral', 'Faculty',
       'Agricultural sciences and natural resources',
       'Biological and biomedical sciences', 'Health sciences', 'Chemistry',
       'Geosciences, atmospheric, and ocean sciences', 'Physics and astronomy',
       'Computer and information sciences', 'Mathematics and statistics',
       'Psychology', 'Anthropology', 'Economics',
       'Political science and government', 'Sociology',
       'Other social sciences',
       'Aerospace, aeronautical, and astronautical engineering',
       'Bioengineering and biomedical engineering', 'Chemical engineering',
       'Civil engineering',
       'Electrical, electronics, and communications engineering',
       'Industrial and manufacturing engineering',
       'Materials science engineering', 'Mechanical engineering',
       'Other engineering', 'Education administration', 'Education research',
       'Teacher education', 'Teaching fields',

In [26]:
pd.Series(' '.join(table['Job Title']).lower().split()).value_counts()

-                                        31257
professor                                24169
of                                       23813
assistant                                22493
adjunct                                  20541
instructor                               19729
faculty                                  16453
art                                      15042
arts                                     12551
and                                      12500
music                                     8690
in                                        7406
philosophy                                6750
&                                         6244
lecturer                                  4931
design                                    4708
humanities                                4585
department                                3836
college                                   3722
studies                                   3422
history                                   3421
pool         

In [22]:
table = faculty_table[(faculty_table['Other humanities and arts']==1)]
table['Number of Detailed Fields of Study'].value_counts()

1     68906
2     14345
3      2399
4       730
5       340
6       240
7       144
8        80
9        25
12       15
10       14
13        2
11        2
19        1
Name: Number of Detailed Fields of Study, dtype: int64

In [21]:
table.sum()

Unnamed: 0                                                     47385426431
Job ID                                                     463931591609185
Year                                                              48786838
Diversity and Inclusion                                                 31
Post-Doctoral                                                         1302
Faculty                                                              22927
Agricultural sciences and natural resources                          24228
Biological and biomedical sciences                                    5774
Health sciences                                                       2291
Chemistry                                                             1674
Geosciences, atmospheric, and ocean sciences                          1006
Physics and astronomy                                                  440
Computer and information sciences                                      424
Mathematics and statistic

In [24]:
table = main_table[main_table['Job Title'].str.contains('kinesiology', case=False, na=False)]
table[['Job ID']].merge(faculty_table, on='Job ID', how='inner').sum()

Job ID                                                     64649695015022
Unnamed: 0                                                     6454187074
Year                                                              6047782
Diversity and Inclusion                                                14
Post-Doctoral                                                           6
Faculty                                                              2525
Agricultural sciences and natural resources                             7
Biological and biomedical sciences                                    988
Health sciences                                                      2271
Chemistry                                                              40
Geosciences, atmospheric, and ocean sciences                            0
Physics and astronomy                                                  52
Computer and information sciences                                     237
Mathematics and statistics            

In [3]:
main_table[main_table['Annual Salary']==200000]['Job ']

,Job ID,Year,Job Title,Occupation,Career Area,Minimum EDU Requirements,Maximum EDU Requirements,Minimum Experience Requirements,Annual Salary,Metropolitan Statistical Area,State,IPEDS Institution Name,IPEDS Sector Name,R1,4-year,2-year,Public,Private,BEA_Zone
598,34338,2011,Office Assistant,Office / Administrative Assistant,Clerical and Administrative,12,-999,1.000000,200000.0,"Spokane-Spokane Valley, WA",Washington,Washington State University,"Public, 4-year or above",1,1,0,1,0,Far West
821,50285,2011,Adjunct Professor Communication Disorders,College Professor / Instructor,Education and Training,-999,-999,-83.250000,200000.0,"New York-Newark-Jersey City, NY-NJ-PA",New Jersey,William Paterson University of New Jersey,"Public, 4-year or above",0,1,0,1,0,Mideast
1211,83067,2011,Leadership Annual Giving Officer,Fundraising / Development Specialist,Business Management and Operations,16,18,-83.250000,200000.0,"Pittsburgh, PA",Pennsylvania,Carnegie Mellon University,"Private not-for-profit, 4-year or above",1,1,0,0,1,Mideast
2619,333432,2011,Pt Staff - Prn - B,Tractor-Trailer Truck Driver (General),Transportation,-999,-999,-83.250000,200000.0,"St. Louis, MO-IL",Missouri,Saint Louis University,"Private not-for-profit, 4-year or above",0,1,0,0,1,Plains
4616,571504,2011,Head Resident II - Graduate Hall Coordinator,Residential Assistant / Advisor,Community and Social Services,10,18,-83.250000,200000.0,"Austin-Round Rock, TX",Texas,The University of Texas System Office,Administrative Unit,0,1,0,1,0,Southwest
4618,571506,2011,Laboratory/Technical Services Supervisor I - L...,Laboratory Analyst,Health Care including Nursing,12,16,3.000000,200000.0,"Austin-Round Rock, TX",Texas,The University of Texas System Office,Administrative Unit,0,1,0,1,0,Southwest
6084,719432,2011,Assistant Hall Director,na,na,16,-999,-83.250000,200000.0,"Knoxville, TN",Tennessee,NaN,"Public, 4-year or above",0,1,0,1,0,Southeast
8842,1118500,2011,Athletic Equipment Attendant I,Entertainment / Recreation Attendant,"Hospitality, Food, and Tourism",-999,-999,-83.250000,200000.0,"Los Angeles-Long Beach-Anaheim, CA",California,University of California-System Administration...,Administrative Unit,0,1,0,1,0,Far West
11149,1408268,2011,Resident Ministry Intern,na,na,16,-999,-83.250000,200000.0,"San Francisco-Oakland-Hayward, CA",California,University of San Francisco,"Private not-for-profit, 4-year or above",0,1,0,0,1,Far West
11840,1457940,2011,Athletic Equipment Attendant I,Entertainment / Recreation Attendant,"Hospitality, Food, and Tourism",-999,-999,-83.250000,200000.0,"Los Angeles-Long Beach-Anaheim, CA",California,University of California-System Administration...,Administrative Unit,0,1,0,1,0,Far West
